# NASA INSIGHT API

In [1]:
import requests
import json
import pandas as pd

In [2]:
with open('../config.json', 'r') as jsonfile:
    data = json.load(jsonfile)
api_key = data['api_key']

In [3]:
url = f'https://api.nasa.gov/insight_weather/?api_key={api_key}&feedtype=json'

In [4]:
def get_mars_weather():
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        print(data)

        sol_keys = data['sol_keys']
        print(f"Latest available Sols: {sol_keys}")

        for sol in sol_keys:
            sol_data = data[sol]
            print(f"\nWeather for Sol {sol}:")
            if 'AT' in sol_data:
                print(f"Average Temperature: {sol_data['AT']['av']}°C")
            else:
                print("Temperature data is not available.")
            
            if 'HWS' in sol_data:
                print(f"Average Wind Speed: {sol_data['HWS']['av']} m/s")
            else:
                print("Wind speed data is not available.")
            
            if 'PRE' in sol_data:
                print(f"Average Atmospheric Pressure: {sol_data['PRE']['av']} Pa")
            else:
                print("Pressure data is not available.")
            
            if 'WD' in sol_data and sol_data['WD'].get('most_common'):
                print(f"Most Common Wind Direction: {sol_data['WD']['most_common']['compass_point']}")
            else:
                print("Wind direction data is not available.")

            print(f"Mars Season: {sol_data['Season']}")
            print(f"First Data UTC: {sol_data['First_UTC']}")
            print(f"Last Data UTC: {sol_data['Last_UTC']}")

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except KeyError as e:
        print(f"Error processing data: {e} - The structure of the data might have changed or it's currently unavailable.")

In [5]:
get_mars_weather()

{'sol_keys': [], 'validity_checks': {'1219': {'AT': {'sol_hours_with_data': [6, 7, 8, 9, 10, 11, 12], 'valid': False}, 'HWS': {'sol_hours_with_data': [6, 7, 8, 9, 10, 11, 12], 'valid': False}, 'PRE': {'sol_hours_with_data': [6, 7, 8, 9, 10, 11, 12], 'valid': False}, 'WD': {'sol_hours_with_data': [6, 7, 8, 9, 10, 11, 12], 'valid': False}}, 'sol_hours_required': 18, 'sols_checked': ['1219']}}
Latest available Sols: []


## Solar flare

In [6]:
# we can change the dates 
solar_flare_url = f'https://api.nasa.gov/DONKI/FLR?startDate=2024-01-01&endDate=2024-02-01&api_key={api_key}'

how it looks like

```python
[{"flrID":"2016-01-01T23:00:00-FLR-001","instruments":[{"displayName":"GOES15: SEM/XRS 1.0-8.0"}],"beginTime":"2016-01-01T23:00Z","peakTime":"2015-01-02T00:10Z","endTime":null,"classType":"M2.3","sourceLocation":"S21W73","activeRegionNum":12473,"linkedEvents":[{"activityID":"2016-01-01T23:12:00-CME-001"},{"activityID":"2016-01-02T02:48:00-SEP-001"},{"activityID":"2016-01-02T04:30:00-SEP-001"}],"link":"https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/FLR/9963/-1"},{"flrID":"2016-01-28T11:48:00-FLR-001","instruments":[{"displayName":"GOES15: SEM/XRS 1.0-8.0"}],"beginTime":"2016-01-28T11:48Z","peakTime":"2016-01-28T12:02Z","endTime":"2016-01-28T12:56Z","classType":"C9.6","sourceLocation":"N03W47","activeRegionNum":12488,"linkedEvents":[{"activityID":"2016-01-28T12:24:00-CME-001"}],"link":"https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/FLR/10122/-1"}]
```

In [7]:
try:
    response = requests.get(solar_flare_url)
    response.raise_for_status()
    data = response.json()
    print(len(data))
except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")

50


In [8]:
data[0]

{'flrID': '2024-01-01T08:33:00-FLR-001',
 'instruments': [{'displayName': 'GOES-P: EXIS 1.0-8.0'}],
 'beginTime': '2024-01-01T08:33Z',
 'peakTime': '2024-01-01T08:54Z',
 'endTime': '2024-01-01T09:04Z',
 'classType': 'M2.3',
 'sourceLocation': 'N03E70',
 'activeRegionNum': 13536,
 'note': '',
 'linkedEvents': None,
 'submissionTime': '2024-01-01T13:28Z',
 'link': 'https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/FLR/28428/-1'}

In [9]:
normalized_data = pd.json_normalize(data)
normalized_data

,flrID,instruments,beginTime,peakTime,endTime,classType,sourceLocation,activeRegionNum,note,linkedEvents,submissionTime,link
0,2024-01-01T08:33:00-FLR-001,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2024-01-01T08:33Z,2024-01-01T08:54Z,2024-01-01T09:04Z,M2.3,N03E70,13536.0,,None,2024-01-01T13:28Z,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...
1,2024-01-01T11:54:00-FLR-001,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2024-01-01T11:54Z,2024-01-01T12:25Z,2024-01-01T12:35Z,M4.7,N03E68,13536.0,,None,2024-01-01T13:24Z,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...
2,2024-01-02T18:02:00-FLR-001,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2024-01-02T18:02Z,2024-01-02T18:30Z,2024-01-02T18:56Z,M1.1,N05E59,13536.0,,[{'activityID': '2024-01-02T19:36:00-CME-001'}],2024-01-02T19:34Z,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...
3,2024-01-04T01:08:00-FLR-001,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2024-01-04T01:08Z,2024-01-04T01:16Z,2024-01-04T01:22Z,M1.1,N02E38,13536.0,,None,2024-01-04T01:44Z,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...
4,2024-01-04T01:22:00-FLR-001,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2024-01-04T01:22Z,2024-01-04T01:55Z,2024-01-04T02:12Z,M3.8,N04E38,13536.0,,None,2024-01-04T03:00Z,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...
5,2024-01-06T05:28:00-FLR-001,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2024-01-06T05:28Z,2024-01-06T05:37Z,2024-01-06T05:41Z,C3.3,N02E01,13536.0,,[{'activityID': '2024-01-06T08:12:00-CME-001'}],2024-01-06T23:44Z,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...
6,2024-01-06T05:41:00-FLR-001,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2024-01-06T05:41Z,2024-01-06T05:47Z,2024-01-06T05:53Z,C3.8,N02E01,13536.0,,[{'activityID': '2024-01-06T08:12:00-CME-001'}],2024-01-06T23:45Z,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...
7,2024-01-06T05:53:00-FLR-001,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2024-01-06T05:53Z,2024-01-06T05:59Z,2024-01-06T06:04Z,C4.1,N02E03,13536.0,,[{'activityID': '2024-01-06T08:12:00-CME-001'}],2024-01-06T23:45Z,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...
8,2024-01-06T06:06:00-FLR-001,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2024-01-06T06:06Z,2024-01-06T06:15Z,2024-01-06T06:21Z,C4.0,N01E11,13536.0,,[{'activityID': '2024-01-06T08:12:00-CME-001'}],2024-01-06T23:45Z,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...
9,2024-01-10T12:39:00-FLR-001,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2024-01-10T12:39Z,2024-01-10T12:55Z,2024-01-10T13:05Z,M1.4,S20E90,13549.0,"At the time of the flare, this flare was assoc...",None,2024-01-16T15:45Z,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...


In [10]:
normalized_data.to_csv('../data/sunflares.csv')

https://www.swpc.noaa.gov/phenomena/solar-flares-radio-blackouts

Features:

- **flrID**: This is an identifier for the flare event.
- **instruments**: These are the tools or instruments used to observe the flare (e.g., GOES15: SEM/XRS 1.0-8.0).
- **beginTime**, peakTime, endTime: These represent the start time, peak time, and end time of the flare event.
- **sourceLocation**: This indicates the solar coordinates (latitude and longitude on the solar surface) where the flare originated.
- **activeRegionNum**: This is the NOAA active region number, which is a unique identifier for the region of the sun where the flare occurred.
- **linkedEvents**: This includes IDs of other space weather events linked to the flare, such as Coronal Mass Ejections (CMEs) or Solar Energetic Particle (SEP) events.
- **link**: This provides a URL to more information about the specific flare event.

NOAA Space Weather Scales

- X-Class Flares: These are the most intense flares. They are major events that can trigger planet-wide radio blackouts and long-lasting radiation storms.

- M-Class Flares: These are medium-sized; they can cause brief radio blackouts that affect Earth's polar regions. Minor radiation storms might occur.

- C-Class Flares: These are small with few noticeable consequences on Earth, except for minor disruptions in polar radio communications.

- B-Class Flares: These are even smaller, with little to no effect on Earth.

- A-Class Flares: These are the smallest flares, which are similar to B-Class in their lack of significant impact on Earth.


The number that follows the letter represents the peak flux (in watts per square meter, W/m²) of x-rays near Earth, as measured in the 1 to 8 angstrom range. For example, an M2.3 class flare has a peak flux between 2x10^-5 and 3x10^-5 W/m².

